## Building Feature for Test Data

.

In [1]:

import datetime, warnings, scipy 
import pandas as pd
import numpy as np
from collections import OrderedDict
from matplotlib.gridspec import GridSpec
from IPython.core.interactiveshell import InteractiveShell
warnings.filterwarnings("ignore")

## Loading Test Data

In [2]:
data=pd.read_csv('clean_flights_test_data.csv',parse_dates= ['fl_date'],encoding='utf-8-sig'
            )   
#data['Month'] = data['Date'].dt.month,data['Day'] = data['Date'].dt.day)
test=data.copy()

In [3]:
data

,fl_date,mkt_unique_carrier,mkt_carrier_fl_num,origin,dest,crs_dep_time,crs_arr_time,crs_elapsed_time,distance
0,2020-01-01,WN,5888,ONT,SFO,1810,1945,95,363
1,2020-01-01,WN,6276,ONT,SFO,1150,1320,90,363
2,2020-01-01,WN,4598,ONT,SJC,2020,2130,70,333
3,2020-01-01,WN,4761,ONT,SJC,1340,1455,75,333
4,2020-01-01,WN,5162,ONT,SJC,915,1035,80,333
...,...,...,...,...,...,...,...,...,...
660551,2020-01-31,DL,4954,DCA,CVG,1859,2041,102,411
660552,2020-01-31,DL,4955,DCA,CVG,1515,1702,107,411
660553,2020-01-31,DL,4956,JFK,BTV,2205,2337,92,266
660554,2020-01-31,DL,4957,ORD,JFK,1035,1356,141,740


### Making the Query to get 1st week of Jan data

In [27]:

test_data=data.query("fl_date >= '2020-01-01' and fl_date <='2020-01-07'")

In [28]:
test3=test_data.copy()

In [29]:
test_data.to_csv('B_firstweek_Jan2020.csv')

## Feature Enginieering for Test Dataset

In [7]:
#according to arrvie delay

 ### 1- Day,Month, Day of week

In [8]:

test3["year"]=test3['fl_date'].dt.year
test3["month"]=test3['fl_date'].dt.month
test3["day"]=test3['fl_date'].dt.day
test3["dayOfWeek"]=test3['fl_date'].dt.dayofweek


In [9]:
test3

,fl_date,mkt_unique_carrier,mkt_carrier_fl_num,origin,dest,crs_dep_time,crs_arr_time,crs_elapsed_time,distance,year,month,day,dayOfWeek
0,2020-01-01,WN,5888,ONT,SFO,1810,1945,95,363,2020,1,1,2
1,2020-01-01,WN,6276,ONT,SFO,1150,1320,90,363,2020,1,1,2
2,2020-01-01,WN,4598,ONT,SJC,2020,2130,70,333,2020,1,1,2
3,2020-01-01,WN,4761,ONT,SJC,1340,1455,75,333,2020,1,1,2
4,2020-01-01,WN,5162,ONT,SJC,915,1035,80,333,2020,1,1,2
...,...,...,...,...,...,...,...,...,...,...,...,...,...
150618,2020-01-07,DL,4813,DTW,JFK,1755,1952,117,509,2020,1,7,1
150619,2020-01-07,DL,4814,GSP,LGA,600,759,119,610,2020,1,7,1
150620,2020-01-07,DL,4815,ATL,XNA,1715,1816,121,589,2020,1,7,1
150621,2020-01-07,DL,4815,XNA,ATL,1851,2145,114,589,2020,1,7,1


### 2- AirLine

In [10]:
Airline_coeff=pd.read_csv('Ariline_coeff.csv')
Airline_coeff

,Unnamed: 0,mkt_unique_carrier,arr_delay,Airline_coeff
0,0,AA,45902,1.759147
1,3,DL,30779,0.850111
2,8,UA,33083,0.988603
3,9,WN,28696,0.724903


In [11]:
for a in Airline_coeff.iterrows():
    print(a)
    test3.loc[test3['mkt_unique_carrier']==a[1]['mkt_unique_carrier'],'Airline_coeff']=\
    a[1]['Airline_coeff']
test3['Airline_coeff'].fillna(0,inplace=True)

(0, Unnamed: 0                  0
mkt_unique_carrier         AA
arr_delay               45902
Airline_coeff         1.75915
Name: 0, dtype: object)
(1, Unnamed: 0                   3
mkt_unique_carrier          DL
arr_delay                30779
Airline_coeff         0.850111
Name: 1, dtype: object)
(2, Unnamed: 0                   8
mkt_unique_carrier          UA
arr_delay                33083
Airline_coeff         0.988603
Name: 2, dtype: object)
(3, Unnamed: 0                   9
mkt_unique_carrier          WN
arr_delay                28696
Airline_coeff         0.724903
Name: 3, dtype: object)


In [12]:
test3['Airline_coeff'].isnull().sum()

0

In [13]:
test3.loc[test3['mkt_unique_carrier']=='AA','Airline_coeff'].count()

39963

## 3- origin airport

In [14]:
test3.dest.str.strip()
test3.origin.str.strip()
Dest=test3.dest.unique()
Orig=test3.origin.unique()


In [15]:
from geopy.geocoders import Nominatim
origloc=[]
for a in Orig:
    address=a+' Airport'   
    geolocator = Nominatim(user_agent="Your_Name")
    location = geolocator.geocode(address)

    #print(location.address)
    try:
        origloc.append([a,location.latitude, location.longitude])
    except:
        print(address)
        origloc.append([a,0, 0])

In [ ]:
#orig_loc=pd.DataFrame(origloc,columns=['Origin_Arprt','Lat','Lon'])
#orig_loc['Lat'].replace({0:48.162246},inplace=True)
#orig_loc['Lon'].replace({0:-103.6313081 },inplace=True)
#orig_loc[orig_loc['Origin_Arprt']=='ISN']

,Origin_Arprt,Lat,Lon
277,ISN,48.162246,-103.631308


In [16]:
orig_loc=pd.DataFrame(origloc,columns=['Origin_Arprt','Lat','Lon'])

In [17]:
for a in orig_loc.iterrows():
    
    test3.loc[test3['origin']==a[1]['Origin_Arprt'],"Lat"]=a[1]["Lat"]
    test3.loc[test3['origin']==a[1]['Origin_Arprt'],"Lon"]=a[1]["Lon"]


In [18]:
test3

,fl_date,mkt_unique_carrier,mkt_carrier_fl_num,origin,dest,crs_dep_time,crs_arr_time,crs_elapsed_time,distance,year,month,day,dayOfWeek,Airline_coeff,Lat,Lon
0,2020-01-01,WN,5888,ONT,SFO,1810,1945,95,363,2020,1,1,2,0.724903,34.054142,-117.599158
1,2020-01-01,WN,6276,ONT,SFO,1150,1320,90,363,2020,1,1,2,0.724903,34.054142,-117.599158
2,2020-01-01,WN,4598,ONT,SJC,2020,2130,70,333,2020,1,1,2,0.724903,34.054142,-117.599158
3,2020-01-01,WN,4761,ONT,SJC,1340,1455,75,333,2020,1,1,2,0.724903,34.054142,-117.599158
4,2020-01-01,WN,5162,ONT,SJC,915,1035,80,333,2020,1,1,2,0.724903,34.054142,-117.599158
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
150618,2020-01-07,DL,4813,DTW,JFK,1755,1952,117,509,2020,1,7,1,0.850111,42.205699,-83.352975
150619,2020-01-07,DL,4814,GSP,LGA,600,759,119,610,2020,1,7,1,0.850111,34.895446,-82.217228
150620,2020-01-07,DL,4815,ATL,XNA,1715,1816,121,589,2020,1,7,1,0.850111,33.637799,-84.429271
150621,2020-01-07,DL,4815,XNA,ATL,1851,2145,114,589,2020,1,7,1,0.850111,36.279810,-94.302590


## 4- by Hour---estimated Departure hour

In [19]:
test3['crs_dep_hour']=test3['crs_dep_time'].apply(lambda x: (x//100))

In [20]:
test3[['crs_dep_hour','crs_dep_time']]


,crs_dep_hour,crs_dep_time
0,18,1810
1,11,1150
2,20,2020
3,13,1340
4,9,915
...,...,...
150618,17,1755
150619,6,600
150620,17,1715
150621,18,1851


### 5- by Hour---estimated arrive hour

In [21]:

test3['crs_arr_hour']=test3['crs_arr_time'].apply(lambda x: (x//100))

### 6- crs_elapsed_time (miniute)

### 7- Distance (Mile)

## Choosing Features

In [23]:
test_f=test3[['distance','crs_dep_hour' , 'crs_arr_hour','crs_elapsed_time',
       'month', 'day', 'dayOfWeek','Airline_coeff', 'Lat', 'Lon']]

In [24]:
test_f

,distance,crs_dep_hour,crs_arr_hour,crs_elapsed_time,month,day,dayOfWeek,Airline_coeff,Lat,Lon
0,363,18,19,95,1,1,2,0.724903,34.054142,-117.599158
1,363,11,13,90,1,1,2,0.724903,34.054142,-117.599158
2,333,20,21,70,1,1,2,0.724903,34.054142,-117.599158
3,333,13,14,75,1,1,2,0.724903,34.054142,-117.599158
4,333,9,10,80,1,1,2,0.724903,34.054142,-117.599158
...,...,...,...,...,...,...,...,...,...,...
150618,509,17,19,117,1,7,1,0.850111,42.205699,-83.352975
150619,610,6,7,119,1,7,1,0.850111,34.895446,-82.217228
150620,589,17,18,121,1,7,1,0.850111,33.637799,-84.429271
150621,589,18,21,114,1,7,1,0.850111,36.279810,-94.302590


### Exporting Features to CSV file

In [25]:
test_f.to_csv('Test_Features.csv')